In [ ]:
from model.model_fast import City

In [ ]:
num_steps  = 2
parameters = {
            'width':     3,
            'height':    3,
}
city = City(num_steps, **parameters)
for time in range(4): # TODO time here and above
    # print(f'Step {time}')
    city.step_fast()
# print(city.newcomer_bid_history)
city.record_run_data_to_file()